In [ ]:
!pip install laspy rasterio

In [ ]:
import laspy
import rasterio
import numpy as np
import os

In [ ]:
def normalize_vegetation_by_dtm(laz_file: str, dtm_tif: str, output_laz: str) -> None:
    """
    Normalizes vegetation point heights using a given DTM.

    Caution: As comments in Line 39-46 shows, for difference LiDAR datasets, the vegetation
             points should be correctly specified.
    
    Parameters
    ----------
    laz_file : str
        Path to the input .laz file (LiDAR point cloud).
    dtm_tif : str
        Path to the DTM (GeoTIFF format).
    output_laz : str
        Path to save the normalized .laz file.

    Returns
    -------
    None
        Saves a new LiDAR file with normalized heights.
    """
    try:
        # Load DTM
        with rasterio.open(dtm_tif) as dtm:
            dtm_data = dtm.read(1)  # Read elevation values
            dtm_transform = dtm.transform  # Get spatial transform
            dtm_bounds = dtm.bounds  # Get extent
            dtm_res = dtm.res  # Resolution
            print(dtm_res)

        # Load LiDAR data
        with laspy.open(laz_file) as las_file:
            las = las_file.read()

        # Here, normally in standard LAZ/LAS files, vegetation points were classified as [3, 4, 5].
        # However, Dutch AHN point clouds donot have vegetation points classified, they are unclassified as [1]
        
        # Extract vegetation points (classification: 3, 4, 5)
        # vegetation_mask = np.isin(las.classification, [3, 4, 5])
        
        # For Dutch AHN case, should be used as below:
        vegetation_mask = np.isin(las.classification, [1])

        # Convert ScaledArrayView to NumPy arrays
        x_veg = np.array(las.x[vegetation_mask])
        y_veg = np.array(las.y[vegetation_mask])
        z_veg = np.array(las.z[vegetation_mask])

        # Convert LiDAR coordinates to raster grid indices
        row_indices = ((dtm_bounds.top - y_veg) / dtm_res[1]).astype(int)
        col_indices = ((x_veg - dtm_bounds.left) / dtm_res[0]).astype(int)

        # Ensure indices are within DTM bounds
        valid_mask = (row_indices >= 0) & (row_indices < dtm_data.shape[0]) & \
                     (col_indices >= 0) & (col_indices < dtm_data.shape[1])

        # Compute normalized heights
        normalized_heights = z_veg[valid_mask] - dtm_data[row_indices[valid_mask], col_indices[valid_mask]]

        # Create new LAS file with normalized heights
        header = laspy.LasHeader(point_format=las.header.point_format, version=las.header.version)
        header.scales = las.header.scales
        header.offsets = las.header.offsets
        
        normalized_las = laspy.LasData(header)
        normalized_las.points = las.points[vegetation_mask][valid_mask]  # Assign filtered points
        normalized_las.z = normalized_heights  # Replace z values with normalized height

        # Save output LAS file
        with laspy.open(output_laz, mode="w", header=header) as las_writer:
            las_writer.write_points(normalized_las.points)

        print(f"Normalized LiDAR file saved: {output_laz}")

    except Exception as e:
        raise Exception(f"Error in normalizing vegetation heights: {e}")


In [ ]:

def normalize_by_dtm(laz_file: str, dtm_tif: str, output_laz: str) -> None:
    """
    Normalizes all LiDAR point heights using a given DTM.

    Parameters
    ----------
    laz_file : str
        Path to the input .laz file (LiDAR point cloud).
    dtm_tif : str
        Path to the DTM (GeoTIFF format).
    output_laz : str
        Path to save the normalized .laz file.

    Returns
    -------
    None
        Saves a new LiDAR file with normalized heights.
    """
    try:
        # Load DTM
        with rasterio.open(dtm_tif) as dtm:
            dtm_data = dtm.read(1)  # Read elevation values
            dtm_transform = dtm.transform  # Get spatial transform
            dtm_bounds = dtm.bounds  # Get extent
            dtm_res = dtm.res  # Resolution
            print(f"DTM resolution: {dtm_res}")

        # Load LiDAR data
        with laspy.open(laz_file) as las_file:
            las = las_file.read()

        # Extract ALL LiDAR points
        x = np.array(las.x)
        y = np.array(las.y)
        z = np.array(las.z)

        # Convert LiDAR coordinates to raster grid indices
        row_indices = ((dtm_bounds.top - y) / dtm_res[1]).astype(int)
        col_indices = ((x - dtm_bounds.left) / dtm_res[0]).astype(int)

        # Ensure indices are within DTM bounds
        valid_mask = (row_indices >= 0) & (row_indices < dtm_data.shape[0]) & \
                     (col_indices >= 0) & (col_indices < dtm_data.shape[1])

        # Compute normalized heights for valid points
        normalized_heights = np.full_like(z, np.nan)  # Initialize with NaNs
        normalized_heights[valid_mask] = z[valid_mask] - dtm_data[row_indices[valid_mask], col_indices[valid_mask]]

        # Create new LAS file with normalized heights
        header = laspy.LasHeader(point_format=las.header.point_format, version=las.header.version)
        header.scales = las.header.scales
        header.offsets = las.header.offsets
        
        normalized_las = laspy.LasData(header)
        normalized_las.points = las.points  # Copy all point attributes
        normalized_las.z = normalized_heights  # Replace Z values with normalized heights

        # Save output LAS file
        with laspy.open(output_laz, mode="w", header=header) as las_writer:
            las_writer.write_points(normalized_las.points)

        print(f"Normalized LiDAR file saved: {output_laz}")

    except Exception as e:
        raise Exception(f"Error in normalizing LiDAR heights: {e}")


In [ ]:

def normalize_by_lowest_point(laz_file: str, output_laz: str, resolution: float = 10.0) -> None:
    """
    Normalizes LiDAR point heights using the lowest point in each grid cell.

    Parameters
    ----------
    laz_file : str
        Path to the input .laz file (LiDAR point cloud).
    output_laz : str
        Path to save the normalized .laz file.
    resolution : float, optional
        Grid resolution for finding the lowest point (default is 10m).

    Returns
    -------
    None
        Saves a new LiDAR file with normalized heights.
    """
    try:
        # Load LiDAR data
        with laspy.open(laz_file) as las_file:
            las = las_file.read()

        # Extract all points (not filtering vegetation)
        x = np.array(las.x)
        y = np.array(las.y)
        z = np.array(las.z)

        # Define grid structure
        min_x, max_x = np.min(x), np.max(x)
        min_y, max_y = np.min(y), np.max(y)

        # Compute grid indices
        x_indices = ((x - min_x) / resolution).astype(int)
        y_indices = ((y - min_y) / resolution).astype(int)

        # Create a dictionary to store the lowest point in each grid cell
        lowest_points = {}

        # Find the lowest elevation per grid cell
        for i in range(len(x)):
            cell = (x_indices[i], y_indices[i])
            if cell not in lowest_points or z[i] < lowest_points[cell]:
                lowest_points[cell] = z[i]

        # Convert dictionary to a NumPy array for fast lookup
        lowest_grid = np.full((np.max(y_indices) + 1, np.max(x_indices) + 1), np.nan)
        for (x_idx, y_idx), min_z in lowest_points.items():
            lowest_grid[y_idx, x_idx] = min_z

        # Normalize heights
        normalized_heights = z - np.array([lowest_grid[y_idx, x_idx] for x_idx, y_idx in zip(x_indices, y_indices)])

        # Create new LAS file with normalized heights
        header = laspy.LasHeader(point_format=las.header.point_format, version=las.header.version)
        header.scales = las.header.scales
        header.offsets = las.header.offsets
        
        normalized_las = laspy.LasData(header)
        normalized_las.points = las.points  # Copy all point attributes
        normalized_las.z = normalized_heights  # Replace Z values with normalized heights

        # Save output LAS file
        with laspy.open(output_laz, mode="w", header=header) as las_writer:
            las_writer.write_points(normalized_las.points)

        print(f"Normalized LiDAR file saved: {output_laz}")

    except Exception as e:
        raise Exception(f"Error in normalizing heights: {e}")


In [ ]:

import os
import pandas as pd

# Define your folder path
folder_path = "/media/jinhu/Data/Data/20250101_AWD/11_VegetationMetrics/24HN1_25/0_Original"

# Get all files in the folder
files = [f for f in os.listdir(folder_path) if f.endswith(('.laz'))]

# Define a simple workflow function
def process_file(file_path):
    try:
        if file_path.endswith('.laz'):
            df = pd.read_csv(file_path)
        elif file_path.endswith('.txt'):
            df = pd.read_csv(file_path, delimiter="\t")  # Adjust delimiter as needed
        else:
            return None

        # Example workflow: Print file name and shape
        print(f"Processing {file_path} - Shape: {df.shape}")
        return df

    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None

# Process each file
for file in files:
    file_path = os.path.join(folder_path, file)
    process_file(file_path)



In [ ]:
# An example of Usage - Normalize all points using the lowest point in grid cells of a given resolution.

input_laz = "../../Datasets/24HN1_25.laz"
output_laz = "../../Datasets/24HN1_25_normalized.laz"
resolution = 1.0 

# Normalize all points using the lowest point in a grid cell of given resolution.
normalize_by_lowest_point(input_laz, output_laz, resolution)

In [ ]:
# Example of Usage - Normalize all points using a DTM file

input_laz = "../../Datasets/24HN1_25.laz"  # Input LiDAR file
dtm_tif = "../../Datasets/24HN1_25_1m_IDW.tif"  # Input DTM file (GeoTIFF)
output_laz = "../../Datasets/normalized_24HN1_25_1m_IDW_all.laz"  # Output file

# Run normalization for all LiDAR points w.r.t. DTM
normalize_by_dtm(input_laz, dtm_tif, output_laz)

In [ ]:
# Example of Usage - Normalize only vegetation points using the generated DTM of a given resolution.

laz_file = "../../Datasets/24HN1_25.laz"  # Input LiDAR file
dtm_tif = "../../Datasets/24HN1_25_1m_IDW.tif"  # Input DTM file (GeoTIFF)
output_laz = "../../Datasets/normalized_24HN1_25_1m_IDW.laz"  # Output file

# Normalize vegetation points only
normalize_vegetation_by_dtm(laz_file, dtm_tif, output_laz)